In [51]:
import os
import numpy as np
import random
from math import log, ceil

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
f = open("fra-eng/fra.txt", "r")

In [14]:
lines = [line.strip("\n").split("\t") for line in f]

In [15]:
lang1 = [x[0] for x in lines]
lang2 = [x[1] for x in lines]

In [67]:
def get_batch(lang1, lang2, alphabet1, alphabet2, num_samples):
    sample = random.sample(range(len(lang1)), num_samples)
    lang1_sample = [lang1[i] for i in sample]
    lang2_sample = [lang2[i] for i in sample]
    maxlen1, maxlen2 = max(map(len, lang1_sample)), max(map(len, lang2_sample))
    lang1_sample = [[alphabet1[i] for i in x.ljust(maxlen1, chr(0))] for x in lang1_sample]
    lang2_sample = [[alphabet2[i] for i in x.ljust(maxlen2, chr(0))] for x in lang2_sample]
    return np.asarray(lang1_sample), np.asarray(lang2_sample)

In [68]:
def build_alphabet_converter(instances):
    chars = set.union(*map(set, instances))
    converter = dict(zip(chars, range(1, len(chars) + 1)))
    converter[chr(0)] = 0
    return converter

In [69]:
alpha1 = build_alphabet_converter(lang1)
alpha2 = build_alphabet_converter(lang2)

In [76]:
x, y = get_batch(lang1, lang2, alpha1, alpha2, 10)
print(x.shape, y.shape)

(10, 49) (10, 53)


In [59]:
def build_alphabet_converter_2d(instances):
    chars = set.union(*map(set, instances))
    max_len = ceil(log(len(chars) + 1, 2))
    binary_strings = [list(map(int, bin(i)[2:].rjust(max_len, '0'))) for i in range(1, len(chars) + 1)]
    converter = dict(zip(chars, binary_strings))
    converter[chr(0)] = [0] * max_len
    return converter